In [1]:
import os
import ipyparallel as ipp

import pandas as pd
import numpy as np
import cplex
from cplex.exceptions import CplexSolverError
from cplex import SparsePair
# from cplex.six.moves import zip
import time
import numba
from ortools.algorithms import pywrapknapsack_solver
from numba import jit
import matplotlib.pyplot as plt
import csv
import pickle
import ctypes
import sys
from sklearn.cluster import KMeans
import binpacking as bp

knapsack = ctypes.CDLL('knapsack.so')
knapsack.knapsack_bnb.restype = ctypes.c_double

In [2]:
rc = ipp.Client()
dv = rc[:]

dv.block=True
dv.execute('import numpy as np')
dv.execute('import binpacking as bp')
dv.execute('import cplex')
dv.execute('import time')
dv.execute('from cplex.exceptions import CplexSolverError')

<AsyncResult: execute:finished>

In [3]:
@ipp.require(bp.binpacking.Master_Sep, bp.binpacking.Master_Kelly)

def ipp_binpack(sector):
    MM, M_var = bp.binpacking.Master_Kelly(w)
    
    penalty = 0.6
    ite = 0
    mt = 0
    st = 0
    solutions = []
    iterations = []
    var_name = []
    objs = []
    coef1 = []
    coef2 = []
    list_basis = []
    var = list(range(len(w)))

    M, var, y, ys = bp.binpacking.Master_Sep(w, sep)
    M.parameters.simplex.tolerances.optimality.set(1e-2)
    
    
    if ipp_sols != 0:
        for sec in range(sep):
            M.variables.add(names=ipp_sols[sec][6], obj=ipp_sols[sec][7])


        for i in range(len(ipp_sols[sec][8])):
            for j in range(len(w)):
                MM.linear_constraints.set_coefficients(int(j),str(ipp_sols[sec][6][i]), float(ipp_sols[sec][8][i][j]))


#         for i in range(len(ipp_sols[sec][9])):
#             MM.linear_constraints.set_coefficients(int(ipp_sols[sec][9][i]),str(str(ipp_sols[sec][6][i])), float(1.0))


    
    for sec in range(sector,sector+1):

        criteria = True
        
        M.set_problem_type(M.problem_type.LP)

        y_fix = list(set(y) - set(list(ys[sec])))
        I_fix = list(set(I) - set(list(Is[sec])))
        # M.objective.set_linear(zip(y_fix, [BigM] * len(y_fix)))
        M.variables.set_upper_bounds(zip(y_fix, np.zeros(len(y_fix))))
        M.variables.set_lower_bounds(zip(y_fix, [0] * len(y_fix)))

        while criteria:

            ite += 1


            if ite % 500 ==0:
                penalty = penalty * 0.6
            if penalty < 0.1:
                penalty = 100



            ct = time.time()

            M.solve()
            
            basis = np.nonzero(M.solution.get_values())[0]
            list_basis.append(np.array(M.variables.get_names())[basis])
            
            
            solutions.append(float(M.solution.get_objective_value()))
            iterations.append(float(cplex._internal._procedural.getitcnt(M._env._e, M._lp)))

            mt += time.time() - ct

            dual = list(M.solution.get_dual_values())

            pi = dual

            pi_ = [dual[i]*penalty for i in I_fix]


            v = pi
            W = int(1000)

            pt = time.time()


            S_obj, sol = bp.binpacking.KnapsackBnB(v, w, W)

            st += time.time() - pt


            if S_obj - 0.00001 > 1.:

                criteria = True
                M.objective.set_linear(list(zip(list(map(lambda x: int(x + len(w)), Is[sec])), pi_)))
                newsub = sol
                label = int('%d0000' % (sec + 1))
                idx = M.variables.get_num() + label
                M.variables.add(names=['x_%d' % (idx)], obj=[1.0])
                M.linear_constraints.set_coefficients(list(zip(list(range(len(w))),
                                                               ['x_%d' % (idx)] * len(var),
                                                               newsub)))
                

                var.append(idx)


            else:
                criteria = False
                # M.write('1.lp')

        if M.solution.get_values(ys[sec]) == [0]*len(ys[sec]):
            break
#     else:
#         continue
#     break

                
        var_name.append('x_%d' % (idx))
        objs.append(0)
        coef1.append(newsub)
#         coef2.append(job + ag)

        
        results = [ite,solutions,iterations,mt,st,M_var,var_name,objs,coef1]
#         results = [ite,solutions,iterations,mt,st,list(np.array(M_var)[indices]),list(np.array(var_name)[indices]),list(np.array(objs)[indices]),list(np.array(coef1)[indices]),list(np.array(coef2)[indices]),list(M.solution.get_dual_values())]
        return results

                

In [4]:
@ipp.require(bp.binpacking.Master_Kelly)
def Separation_CG(binpacking,ite,w,sep):
    
    solutions = []
    iterations = []
    ite = 0
    ite2 = 0
    mt = 0
    st = 0
    

    start= time.time()
    ite_all  = 0
    
    global var
    var = list(range(len(w)))
    I = list(range(len(w)))
    
    clustering = False

    Is = list(np.array_split(I, sep))
        
    ########### parallel execution #######
    dv['var'] = var
    dv['w'] = w
    dv['sep'] = sep
    dv['I'] = I
    dv['Is'] = Is
    dv['ipp_sols'] = 0

    sector = list(range(sep))

    ipp_sols = dv.map(ipp_binpack, sector)    
    step1 = time.time()-start
    print('step1:', step1)
    
    atime = time.time()

    MM, M_var = binpacking.Master_Kelly(w)
    MM.set_problem_type(MM.problem_type.LP)
    
#     return MM, ipp_sols
    
    for sec in sector:
        
        ite += ipp_sols[sec][0]
        mt += ipp_sols[sec][3]
        st += ipp_sols[sec][4]
        solutions += ipp_sols[sec][1]
        iterations += ipp_sols[sec][2]
        MM.variables.add(names=ipp_sols[sec][6], obj=ipp_sols[sec][7])

        for i in range(len(ipp_sols[sec][8])):
            for j in range(len(w)):
                MM.linear_constraints.set_coefficients(int(j),str(ipp_sols[sec][6][i]), float(ipp_sols[sec][8][i][j]))


#         for i in range(len(ipp_sols[sec][9])):
#             MM.linear_constraints.set_coefficients(int(ipp_sols[sec][9][i]),str(str(ipp_sols[sec][6][i])), float(1.0))

    
    step2 = time.time()-atime
    print('step2:', step2)
    ct = time.time()

#     MM.parameters.simplex.tolerances.optimality.set(1e-2)
    MM.solve()
    mt += time.time() - ct
    solutions.append(float(MM.solution.get_objective_value()))
    iterations.append(float(cplex._internal._procedural.getitcnt(MM._env._e, MM._lp)))


    
    
    #### Kelly Column Generation ######
    
    
    Kelly_result = binpacking.Kelly_CG(MM, 0.00001, M_var)
 
    ite2 += Kelly_result[0]
    mt += Kelly_result[1]
    st += Kelly_result[2]
    solutions += Kelly_result[3]
    iterations += Kelly_result[4]
    print('step3:', time.time() - ct)
    
    tt = time.time() - start
    print('total:',tt)
    
    Separation_result = [

        'Separation', ite,ite2, mt, st, tt, mt / (st + mt),
        np.average(np.array(iterations)),MM

    ]
    
    return Separation_result


In [5]:
def result_table(results):
    table = pd.concat(results, axis=1)
    name = ['method', '#S_iter','#K_iter', 'M', 'S', 'total', 'M_per', 'pivot_iteration','MM']

    table = table.transpose()

    table.columns = name

    table.drop(['S','MM'], axis=1, inplace=True)

    return table

In [6]:
PS = {}
sep = 2
clustering = False
problems = ['']
# problems = ['d05100','d10100','d10200','d20100','d20200','e05100','e10100','e10200','e20100','e20200']
# problems = ['d10200']


# for problem in problems:
#     print(problem)
#     gap = GAP.GAP(
#         problem=problem, sep=sep, timelimit=3600, clustering=clustering
#     )
#     ite = 0
#     Separation_result = Separation_CG(gap,ite,gap.agent,gap.job,gap.a,gap.b,gap.c,sep)
#     PS[problem] = Separation_result
    
for prob in range(20):

    bin = bp.binpacking(
        prob_num=prob, type = 't501', sep=sep
    )

    print(prob)
    ite = 0
    
    Separation_result = Separation_CG(bin,ite,bin.w,sep)
    PS[prob] = Separation_result

0
step1: 4.908117055892944
step2: 0.04040980339050293
step3: 1.3771052360534668
total: 6.326208114624023
1
step1: 4.4216649532318115
step2: 0.04450583457946777
step3: 1.227813720703125
total: 5.694378852844238
2
step1: 43.94017195701599
step2: 0.04184889793395996
step3: 1.8028900623321533
total: 45.78522181510925
3
step1: 6.66003680229187
step2: 0.038298845291137695
step3: 1.6618871688842773
total: 8.360658884048462
4
step1: 15.923718929290771
step2: 0.04203009605407715
step3: 1.3367798328399658
total: 17.303020000457764
5
step1: 10.590816020965576
step2: 0.03944706916809082
step3: 1.3498609066009521
total: 11.9811110496521
6
step1: 4.822760105133057
step2: 0.06045699119567871
step3: 1.6827189922332764
total: 6.566726922988892
7
step1: 5.775629758834839
step2: 0.03716111183166504
step3: 1.446516990661621
total: 7.259800672531128
8
step1: 11.152432918548584
step2: 0.03962898254394531
step3: 1.6909630298614502
total: 12.883328914642334
9
step1: 123.51627588272095
step2: 0.050858974456787

In [12]:
PS = {}
sep = 2
clustering = False
problems = ['']
# problems = ['d05100','d10100','d10200','d20100','d20200','e05100','e10100','e10200','e20100','e20200']
# problems = ['d10200']


# for problem in problems:
#     print(problem)
#     gap = GAP.GAP(
#         problem=problem, sep=sep, timelimit=3600, clustering=clustering
#     )
#     ite = 0
#     Separation_result = Separation_CG(gap,ite,gap.agent,gap.job,gap.a,gap.b,gap.c,sep)
#     PS[problem] = Separation_result
    
for prob in range(20):

    bin = bp.binpacking(
        prob_num=prob, type = 't501', sep=sep
    )

    print(prob)
    ite = 0
    
    Separation_result = Separation_CG(bin,ite,bin.w,sep)
    PS[prob] = Separation_result

0
step1: 4.454860687255859
step2: 0.034039974212646484
step3: 6.734695911407471
total: 11.223982810974121
1
step1: 3.8524510860443115
step2: 0.06237912178039551
step3: 10.056503772735596
total: 13.971672058105469
2
step1: 40.766247034072876
step2: 0.034696102142333984
step3: 5.837723255157471
total: 46.63897395133972
3
step1: 6.166554927825928
step2: 0.03409910202026367
step3: 7.2227020263671875
total: 13.423612117767334
4
step1: 15.537399053573608
step2: 0.0442047119140625
step3: 6.399587631225586
total: 21.981518745422363
5
step1: 9.686367988586426
step2: 0.03381800651550293
step3: 8.110427856445312
total: 17.83097791671753
6
step1: 4.550123929977417
step2: 0.03373885154724121
step3: 406.2089412212372
total: 410.7932028770447
7
step1: 5.364219903945923
step2: 0.03375697135925293
step3: 6.031386137008667
total: 11.429717063903809
8
step1: 10.78801417350769
step2: 0.03366589546203613
step3: 6.2809741497039795
total: 17.102972269058228
9
step1: 119.2366669178009
step2: 0.041515111923217

In [13]:
results = [pd.DataFrame(PS)]
result_table(results).to_csv('t501_parallel.csv')
result_table(results)

,method,#S_iter,#K_iter,M,total,M_per,pivot_iteration
0,Separation,770,1073,6.67345,11.224,0.887295,40.8401
1,Separation,751,1078,7.24812,13.9717,0.687158,42.9257
2,Separation,3399,889,13.4807,46.639,0.474771,18.4769
3,Separation,1170,1043,7.47191,13.4236,0.824183,36.0944
4,Separation,1868,1013,8.89476,21.9815,0.875139,24.828
5,Separation,1585,1147,9.45653,17.831,0.877965,33.0373
6,Separation,779,957,5.87105,410.793,0.0144198,38.183
7,Separation,991,960,6.48965,11.4297,0.881755,38.1167
8,Separation,1389,984,7.734,17.103,0.878643,31.5815
9,Separation,6515,1102,49.6554,126.215,0.935773,11.2693
